# Disease Extraction (NER problem) 
https://datahack.analyticsvidhya.com/contest/innoplexus-online-hiring-hackathon-saving-lives-wi/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Imports

In [ ]:
import os
os.chdir('/content/drive/MyDrive/IR-pro/Disease-Extraction')

In [ ]:
!pwd

/content/drive/MyDrive/IR-pro/Disease-Extraction


In [ ]:
!pip install scispacy

     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 13.0MB 290kB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 10.4MB 2.1MB/s 
     |████████████████████████████████| 194kB 41.9MB/s 
     |████████████████████████████████| 1.1MB 34.2MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
import pandas as pd
import numpy as np

from collections import namedtuple
from itertools import repeat
from copy import deepcopy

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import spacy
import scispacy
from spacy.util import minibatch, compounding

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook as tqdm

import gc
import random
from pathlib import Path

from multiprocessing import Pool
import multiprocessing
N_CORES = multiprocessing.cpu_count()
print('Number of CPUs: ', N_CORES)

Number of CPUs:  2


#### Evaluation Metric

In [ ]:
def collect_named_entities(tokens): # Helper Function for score calculation
    """
    Creates a list of Entity named-tuples, storing the entity type and the start and end
    offsets of the entity.
    :param tokens: a list of labels
    :return: a list of Entity named-tuples
    """
    Entity = namedtuple("Entity", "e_type start_offset end_offset")
    named_entities = []
    start_offset = None
    end_offset = None
    ent_type = None

    for offset, token_tag in enumerate(tokens):

        if token_tag == 'O':
            if ent_type is not None and start_offset is not None:
                end_offset = offset - 1
                named_entities.append(Entity(ent_type, start_offset, end_offset))
                start_offset = None
                end_offset = None
                ent_type = None

        elif ent_type is None:
            ent_type = token_tag[2:]
            start_offset = offset

        elif ent_type != token_tag[2:] or (ent_type == token_tag[2:] and token_tag[:1] == 'B'):

            end_offset = offset - 1
            named_entities.append(Entity(ent_type, start_offset, end_offset))

            # start of a new entity
            ent_type = token_tag[2:]
            start_offset = offset
            end_offset = None

    # catches an entity that goes up until the last token
    if ent_type and start_offset and end_offset is None:
        named_entities.append(Entity(ent_type, start_offset, len(tokens)-1))

    return named_entities

def compute_metrics(true_named_entities, pred_named_entities): # Helper Function for score calculation
    eval_metrics = {'correct': 0, 'partial': 0, 'missed': 0, 'spurius': 0}
    target_tags_no_schema = ['indications']

    # overall results
    evaluation = {'partial': deepcopy(eval_metrics)}


    true_which_overlapped_with_pred = []  # keep track of entities that overlapped

    # go through each predicted named-entity
    for pred in pred_named_entities:
        found_overlap = False

        # check if there's an exact match, i.e.: boundary and entity type match
        if pred in true_named_entities:
            true_which_overlapped_with_pred.append(pred)
            evaluation['partial']['correct'] += 1

        else:

            # check for overlaps with any of the true entities
            for true in true_named_entities:

                
                # 2. check for an overlap i.e. not exact boundary match, with true entities
                if pred.start_offset <= true.end_offset and true.start_offset <= pred.end_offset:

                    true_which_overlapped_with_pred.append(true)

                    evaluation['partial']['partial'] += 1

                    found_overlap = True
                    break

            # count spurius (i.e., False Positive) entities
            if not found_overlap:
                # overall results
                evaluation['partial']['spurius'] += 1

    # count missed entities (i.e. False Negative)
    for true in true_named_entities:
        if true in true_which_overlapped_with_pred:
            continue
        else:
            # overall results
            evaluation['partial']['missed'] += 1

    # Compute 'possible', 'actual'
    for eval_type in ['partial']:

        correct = evaluation[eval_type]['correct']
        partial = evaluation[eval_type]['partial']
        missed = evaluation[eval_type]['missed']
        spurius = evaluation[eval_type]['spurius']

        # possible: nr. annotations in the gold-standard which contribute to the final score
        evaluation[eval_type]['possible'] = correct + partial + missed

        # actual: number of annotations produced by the NER system
        evaluation[eval_type]['actual'] = correct + partial + spurius

        actual = evaluation[eval_type]['actual']
        possible = evaluation[eval_type]['possible']

    return evaluation

def list_converter(df): # Helper Function for score calculation
    keys, values = df.sort_values('Sent_ID_x').values.T
    ukeys, index = np.unique(keys,True)
    lists = [list(array) for array in np.split(values,index[1:])]
    return lists

# ideal and pred respectively represent dataframes containing actual labels and predictions for the set of sentences in the test data. 
# It has the same format as the sample submission (id, Sent_ID, tag)

def calculate_score(ideal, pred): # Calculates the final F1 Score

    merged = ideal.merge(pred, on = "id", how="inner").drop(['Sent_ID_y'],axis = 1)
    
    
    # The scores are calculated sentence wise and then aggregated to calculate the overall score, for this
    # List converter function groups the labels by sentence to generate a list of lists with each inner list representing a sentence in sequence
    ideal_ = list_converter(merged.drop(['id','tag_y'],axis = 1))
    pred_ = list_converter(merged.drop(['id','tag_x'],axis = 1))

    metrics_results = {'correct': 0, 'partial': 0,
                   'missed': 0, 'spurius': 0, 'possible': 0, 'actual': 0}

    results = {'partial': deepcopy(metrics_results)}


    for true_ents, pred_ents in zip(ideal_, pred_):    
    # compute results for one sentence
        tmp_results = compute_metrics(collect_named_entities(true_ents),collect_named_entities(pred_ents))
    
    # aggregate overall results
        for eval_schema in results.keys():
            for metric in metrics_results.keys():
                results[eval_schema][metric] += tmp_results[eval_schema][metric]
    correct = results['partial']['correct']
    partial = results['partial']['partial']
    missed = results['partial']['missed']
    spurius = results['partial']['spurius']
    actual = results['partial']['actual']
    possible = results['partial']['possible']


    precision = (correct + 0.5 * partial) / actual if actual > 0 else 0
    recall = (correct + 0.5 * partial) / possible if possible > 0 else 0


    score = (2 * precision * recall)/(precision + recall) if (precision + recall) >0 else 0
    
    # final score
    return score

#### Load Data and analyse

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/IR-pro/Disease-Extraction/data/train.csv')

In [ ]:
train_df.head()

,id,Doc_ID,Sent_ID,Word,tag
0,1,1,1,Obesity,O
1,2,1,1,in,O
2,3,1,1,Low-,O
3,4,1,1,and,O
4,5,1,1,Middle-Income,O


In [ ]:
train_df.shape

(4543833, 5)

In [ ]:
train_df['Doc_ID'].nunique()

30000

In [ ]:
train_df['Sent_ID'].nunique()

191282

In [ ]:
train_df['id'].nunique()

4543833

In [ ]:
train_df['tag'].value_counts()

O                4446206
B-indications      53003
I-indications      44624
Name: tag, dtype: int64

We have total 30000 documents and 191282 sentances in total.

We have total 4543833 unique id, which means each record is assigned unique id.

#### Sci-spacy demo

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bc5cdr_md-0.3.0.tar.gz

     |████████████████████████████████| 79.8MB 112kB/s 
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.3.0-cp36-none-any.whl size=79880571 sha256=84c90f1d57250f40beba3729a093e22210960020ea744ce9b7e7269be0d6810c
  Stored in directory: /root/.cache/pip/wheels/c9/22/69/059a5d6de407bcd8c7fb65098f02c9b874171b6549f74b882e
Successfully built en-ner-bc5cdr-md


In [ ]:
import en_ner_bc5cdr_md
nlp = en_ner_bc5cdr_md.load()

In [ ]:
text = """Obesity in Low- and Middle-Income Countries : Burden , Drivers , and Emerging Challenges . We have reviewed the distinctive features of excess weight , its causes , and related prevention and management efforts , as well as data gaps and recommendations for future research in low- and middle-income countries ( LMICs ) . Obesity is rising in every region of the world , and no country has been successful at reversing the epidemic once it has begun . In LMICs , overweight is higher in women compared with men , in urban compared with rural settings , and in older compared with younger individuals ; however , the urban-rural overweight differential is shrinking in many countries . Overweight occurs alongside persistent burdens of underweight in LMICs , especially in young women . Changes in the global diet and physical activity are among the hypothesized leading contributors to obesity . Emerging risk factors include environmental contaminants , chronic psychosocial stress , neuroendocrine dysregulation , and genetic/epigenetic mechanisms . Data on effective strategies to prevent the onset of obesity in LMICs or elsewhere are limited . Expanding the research in this area is a key priority and has important possibilities for reverse innovation that may also inform interventions in high-income countries . """
doc = nlp(text)

In [ ]:
[(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]

[('Obesity', 0, 7, 'DISEASE'),
 ('Low-', 11, 15, 'CHEMICAL'),
 ('gaps', 229, 233, 'DISEASE'),
 ('Obesity', 322, 329, 'DISEASE'),
 ('overweight', 463, 473, 'DISEASE'),
 ('obesity', 886, 893, 'DISEASE'),
 ('obesity', 1105, 1112, 'DISEASE')]

In [ ]:
# [(X, X.ent_iob_, X.ent_type_) for X in doc]

### Create Spacy NER prediction dataset

create train and validation set.

We are doing this based on if a doc has any entity in it or not.

We will create stratified train and test split based on that.

In [ ]:
has_ent = train_df.groupby('Doc_ID')['tag'].apply(lambda x: 'B-indications' in x.values).reset_index()

In [ ]:
has_ent.head()

,Doc_ID,tag
0,1,True
1,2,True
2,3,True
3,4,False
4,5,False


In [ ]:
has_ent.shape

(30000, 2)

Get train Doc ids and validation Doc ids

In [ ]:
trn_doc_ids, tst_doc_ids = train_test_split(has_ent['Doc_ID'].values, test_size = 0.33, stratify=has_ent['tag'].values, random_state=0)

In [ ]:
print(len(trn_doc_ids), " training documents.")

20100  training documents.


In [ ]:
print(len(tst_doc_ids), " test documents.")

9900  test documents.


In [ ]:
test_df = train_df[train_df['Doc_ID'].isin(tst_doc_ids)].reset_index(drop=True)
train_df = train_df[train_df['Doc_ID'].isin(trn_doc_ids)].reset_index(drop=True)

In [ ]:
test_df.head()

,id,Doc_ID,Sent_ID,Word,tag
0,212,2,10,MICROCEPHALIA,B-indications
1,213,2,10,VERA,I-indications
2,214,3,11,Excellent,O
3,215,3,11,reproducibility,O
4,216,3,11,of,O


We want to train our model on document level and also predict on document level, that's why we need to group data by Doc_ID.

The reason behid this is that in training out model can use context words and learn better.

Same goes for prediction. If we just single word for prediction, then model doesn't know its context words, and model can't predict better.


Main problem we face in using using spacy for prediction is that it takes whole document, do tokenization on its own and give us IOB (inside, outside, begining) prediction on tokens it has generated. Here we might face mismatch between tokens of spacy and our own, we need IOB predictions on our own tokens.

That's why we have created a function which gives use prediction based on word location in document.

In [ ]:
# Function for creating test data for prediction
def gb_ops_test(df):
    ids = df['id'].tolist()
    st_inds = []
    doc = ""
    ls_ind = 0
    for w in df['Word']:
        st_inds.append(ls_ind)
        w_len = len(str(w))
        ls_ind = ls_ind + w_len + 1
        doc = doc + str(w) + " "
    return pd.Series(dict(ids = ids, st_inds = st_inds, doc = doc))

In [ ]:
test_data = test_df.groupby('Doc_ID').apply(gb_ops_test)

In [ ]:
test_data.head(5)

,ids,st_inds,doc
Doc_ID,,,
2,"[212, 213]","[0, 14]",MICROCEPHALIA VERA
3,"[214, 215, 216, 217, 218, 219, 220, 221, 222, ...","[0, 10, 26, 29, 35, 43, 52, 60, 63, 70, 75, 89...",Excellent reproducibility of laser speckle con...
4,"[519, 520, 521, 522, 523, 524, 525, 526, 527, ...","[0, 9, 19, 26, 29, 44, 47, 51, 63, 68, 74]",Positive inotropic action of cholinesterase on...
6,"[830, 831, 832, 833, 834, 835, 836, 837, 838, ...","[0, 15, 20, 29, 37, 39, 44, 46, 48, 51, 60, 65...",Self-assembled drug delivery systems . Part 8 ...
7,"[1066, 1067, 1068, 1069, 1070, 1071, 1072, 107...","[0, 12, 16, 23, 34, 37, 44, 54, 57, 77, 87, 92...",Hyperphagia and leptin resistance in tissue in...


Here, ids contains word ids in doc, st_inds has starting index of all words which are in doc. and doc is text document.

ids and st_inds has samesize.

In [ ]:
#ans = {'B': [], 'I': []}
#type(ans)

#### Make a benchmark model using pre trained Sci Spacy 'en_ner_bc5cdr_md' model

Which is a NER for DISEASE and CHEMICHAL entities.

In [ ]:
# function for prediction on document and returns 'B' and 'I' word ids in our dataset

def get_tag(ids, st_inds, doc, nlp_obj):
    ids = pd.Series(ids)
    st_inds = pd.Series(st_inds)

    doc = nlp_obj(str(doc))
    out = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]

    ans = {'B': [], 'I': []}

    for o in out:
        if o[3] == 'DISEASE':
            inss = st_inds[(st_inds >= o[1]) & (st_inds <= o[2])].index.tolist()
            if (st_inds == o[1]).sum() == 0:
                inss = [(st_inds[st_inds > o[1]].index[0] - 1)] + inss
            w_ids = ids.iloc[inss].tolist()
            ans['B'].append(w_ids[0])
            ans['I'].extend(w_ids[1:])
    return ans

def get_B_I_ids(temp_data):
    B_pred_ids = []
    I_pred_ids = []
    for d in temp_data:
        B_pred_ids.extend(d['B'])
        I_pred_ids.extend(d['I'])
        
    return B_pred_ids, I_pred_ids

def make_prediction(test_df, B_pred_ids, I_pred_ids):
    ans = test_df[['id', 'Sent_ID']].copy()
    ans['tag'] = 'O'
    ans.loc[ans['id'].isin(B_pred_ids), 'tag'] = 'B-indications'
    ans.loc[ans['id'].isin(I_pred_ids), 'tag'] = 'I-indications'
    return ans

In [ ]:
import time

In [ ]:
# spacy NLP object
nlp = spacy.load("en_ner_bc5cdr_md")

# function to put nlp object in get_tag function
def mp_get_tag(ids, st_inds, doc):
    return get_tag(ids, st_inds, doc, nlp)

## Multiprocess code
t1 = time.time()
p = Pool(N_CORES)
temp_data = list(p.starmap(mp_get_tag, zip(list(test_data['ids'].values), list(test_data['st_inds'].values), list(test_data['doc'].values))))
p.close()
p.join()
p.terminate()
print((time.time() - t1)/60, " minutes")

4.601437032222748  minutes


In [ ]:
B_pred_ids, I_pred_ids = get_B_I_ids(temp_data)
test_pred_bm = make_prediction(test_df, B_pred_ids, I_pred_ids)

In [ ]:
test_pred_bm.head(180)

,id,Sent_ID,tag
0,212,10,O
1,213,10,O
2,214,11,O
3,215,11,O
4,216,11,O
...,...,...,...
175,387,18,O
176,388,18,O
177,389,18,O
178,390,18,O


In [ ]:
test_pred_bm['tag'].value_counts()

O                1451370
B-indications      28439
I-indications      16967
Name: tag, dtype: int64

In [ ]:
calculate_score(test_df[['id', 'Sent_ID', 'tag']], test_pred_bm)

0.5213439101179085


We can Update (further train) this sci spacy model on our train dataset to increase score 

### Create Spacy train dataset

In [ ]:
def create_trn_data(df):
    ent_pres = False
    doc = ""
    ents = []
    new_ind = 0
    
    for i, r in df.iterrows():
        w_len = len(str(r['Word']))
        doc = doc + str(r['Word']) + " "
        
        if r['tag'] == 'O':
            new_ind = new_ind + w_len + 1
            
        if r['tag'] == 'B-indications':
            st = new_ind
            en = st + w_len
            ents.append((st, en, 'DISEASE'))
            new_ind = en + 1
            
        if r['tag'] == 'I-indications':
            en = new_ind + w_len
            st = ents[-1][0]
            ents = ents[0:-1]
            ents.append((st, en, 'DISEASE'))
            new_ind = en + 1
    
    if ents:
        ent_pres = True
    
    out = (doc, {"entities": ents})
    return pd.Series(dict(trn_data = out, ent_pres = ent_pres))


In [ ]:
train_data = train_df.groupby('Sent_ID').apply(create_trn_data)

In [ ]:
train_data.head(10)

,trn_data,ent_pres
Sent_ID,,
1,(Obesity in Low- and Middle-Income Countries :...,False
2,(We have reviewed the distinctive features of ...,False
3,(Obesity is rising in every region of the worl...,False
4,"(In LMICs , overweight is higher in women comp...",False
5,(Overweight occurs alongside persistent burden...,False
6,(Changes in the global diet and physical activ...,False
7,(Emerging risk factors include environmental c...,False
8,(Data on effective strategies to prevent the o...,True
9,(Expanding the research in this area is a key ...,False


In [ ]:
train_data.iloc[7]

trn_data    (Data on effective strategies to prevent the o...
ent_pres                                                 True
Name: 8, dtype: object

In [ ]:
train_data.iloc[1]["trn_data"]

### Train sci spacy model

In [ ]:
def trn_model(model= None, output_dir=None, n_iter=10):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in tqdm(batches):
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.3,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)
    
            # save model to output directory
            if output_dir is not None:
                output_dir = Path(output_dir)
                if not output_dir.exists():
                    output_dir.mkdir()
                nlp.to_disk(output_dir)
                print("Saved model to", output_dir)

    return nlp

In [ ]:
TRAIN_DATA = train_data['trn_data'].tolist()

In [ ]:
MODEL = "en_ner_bc5cdr_md"
OUT_DIR = "/content/drive/MyDrive/IR-pro/Disease-Extraction/code/model_final"

In [ ]:
nlp1 = trn_model(MODEL, OUT_DIR, 5)

Loaded model 'en_ner_bc5cdr_md'


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Losses {'ner': 39699.39966408337}
Saved model to /content/drive/MyDrive/IR-pro/Disease-Extraction/code/model_final



Losses {'ner': 32803.490870815134}
Saved model to /content/drive/MyDrive/IR-pro/Disease-Extraction/code/model_final



Losses {'ner': 29189.849480660396}
Saved model to /content/drive/MyDrive/IR-pro/Disease-Extraction/code/model_final



Losses {'ner': 26935.696405625004}
Saved model to /content/drive/MyDrive/IR-pro/Disease-Extraction/code/model_final



Losses {'ner': 25338.343921546584}
Saved model to /content/drive/MyDrive/IR-pro/Disease-Extraction/code/model_final


#### Now let's predict on test set using this new updated model

In [ ]:
# function to put nlp object in get_tag function
def mp_get_tag(ids, st_inds, doc):
    return get_tag(ids, st_inds, doc, nlp1)

## Multiprocess code
t1 = time.time()
p = Pool(N_CORES)
temp_data = list(p.starmap(mp_get_tag, zip(list(test_data['ids'].values), list(test_data['st_inds'].values), list(test_data['doc'].values))))
p.close()
p.join()
p.terminate()
print((time.time() - t1)/60, " minutes")

4.452655212084452  minutes


In [ ]:
test_data.head()

,ids,st_inds,doc
Doc_ID,,,
2,"[212, 213]","[0, 14]",MICROCEPHALIA VERA
3,"[214, 215, 216, 217, 218, 219, 220, 221, 222, ...","[0, 10, 26, 29, 35, 43, 52, 60, 63, 70, 75, 89...",Excellent reproducibility of laser speckle con...
4,"[519, 520, 521, 522, 523, 524, 525, 526, 527, ...","[0, 9, 19, 26, 29, 44, 47, 51, 63, 68, 74]",Positive inotropic action of cholinesterase on...
6,"[830, 831, 832, 833, 834, 835, 836, 837, 838, ...","[0, 15, 20, 29, 37, 39, 44, 46, 48, 51, 60, 65...",Self-assembled drug delivery systems . Part 8 ...
7,"[1066, 1067, 1068, 1069, 1070, 1071, 1072, 107...","[0, 12, 16, 23, 34, 37, 44, 54, 57, 77, 87, 92...",Hyperphagia and leptin resistance in tissue in...


In [ ]:
B_pred_ids, I_pred_ids = get_B_I_ids(temp_data)
test_pred_final = make_prediction(test_df, B_pred_ids, I_pred_ids)

In [ ]:
''''for t in test_pred_final['tag']:
    if t!= 'O':
        print(test_pred_final['id'],t)'''

In [ ]:
test_pred_final.tail()

,id,Sent_ID,tag
1496771,4543366,191263,O
1496772,4543367,191263,O
1496773,4543368,191263,O
1496774,4543369,191263,O
1496775,4543370,191263,O


In [ ]:
test_pred_final['tag'].value_counts()

O                1470599
B-indications      14836
I-indications      11341
Name: tag, dtype: int64

In [ ]:
calculate_score(test_df[['id', 'Sent_ID', 'tag']], test_pred_final)

0.7815614274490345

In [ ]:
#test_pred_final.loc["B-indications"]
fin_lst=test_pred_final.values.tolist()
#print(fin_lst[:10])
for i in fin_lst:
    if i[2]!='O':
        print(i)
        print()


In [ ]:
tst=test_data.values.tolist()
#print(tst[0])
for i in tst:
    if 4083144 in i[0]:
        print(i[2])
        text=i[2]
doc = nlp1(text)
[(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]

In [ ]:
text1="The Italian Diabetes and Exercise Study 2 ( IDES-2 ) : a long-term behavioral intervention for adoption and maintenance of a physically active lifestyle . Physical activity ( PA ) /exercise have become an integral part of the management of type 2 diabetes mellitus ( T2DM ) . However , current guidelines are difficult to put into action in this population due to a number of barriers , especially the lack of acceptable , feasible , and validated behavioral intervention strategies . The present manuscript reports the rationale , study design and methods , and design considerations of the Italian Diabetes and Exercise Study ( IDES ) -2 , a randomized controlled trial testing the efficacy of a behavior change strategy in increasing total daily PA and reducing sedentary time ( SED-time ) in patients with T2DM . Starting 7 January 2014 , the IDES_2 began enrolling 300 patients with known T2DM of at least 1-year duration in three tertiary referral outpatient Diabetes Clinics in Rome . Additional requirements are age 40 to 80 years , body mass index 27 to 40 kg/m ( 2 ) , sedentary lifestyle , and physically inactive for at least 6 months , ability to walk 1.6 km without assistance , and eligibility after cardiovascular evaluation . Patients are randomized by center and within each center , by age and type of diabetes treatment to either the intervention or the control group . Patients in the intervention ( INT ) group ( n = 150 ) receive theoretical and practical exercise counseling consisting of aggregated behavior change techniques ( one individual theoretical counseling session plus eight twice-a-week individual theoretical and practical exercise counseling sessions ) once a year for 3 years . Patients in the control ( CON ) group ( n = 150 ) , receive standard care , including general physician recommendations for daily PA . The primary outcomes are total daily PA and SED-time , as measured objectively by the use of an accelerometer . Secondary outcomes include physical fitness , modifiable cardiovascular risk factors , musculoskeletal disturbances , well-being/depression , and health-related quality of life . The behavioral intervention strategy tested in the IDES_2 is based on solid theoretical grounds and uses several behavioral change techniques , two factors which were found to improve effectiveness of behavioral intervention . In addition , physicians and exercise specialists have been specifically trained for counselling/prescribing and supervising PA/exercise , respectively , in subjects suffering from metabolic disorders . Finally , the large sample size , the long study duration , and the objective measurement of PA allow statistically significant and scientifically robust conclusions to be drawn on the feasibility and efficacy of this intervention in T2DM patients . ClinicalTrials.gov ; NCT01600937 ; 10 October 2012 . "
text="Dissociation of acute and chronic intermittent phencyclidine-induced performance deficits in the 5-choice serial reaction time task : influence of clozapine . Cognitive deficits are a core feature of schizophrenia that respond minimally to existing drugs . PCP is commonly used to model schizophrenia-like deficits preclinically although different dosing protocols may affect different domains . Here we characterise the acute , and chronic intermittent effects of PCP in the 5-choice serial reaction time task ( 5-CSRTT ) in rats , and assess the effects of clozapine . In a novel approach , we also assess the effects of increased inhibitory load and conduct clinically relevant signal detection analysis ( SDA ) . The effects of acute and repeated PCP ( 2.58 mg/kg ) treatment on attentional processes and inhibitory control were assessed during and following the chronic treatment regime in the presence or absence of chronic clozapine ( 20 mg/kg/day ) . Thirty minutes post-PCP injection , there was an increase in anticipatory responding which disappeared after 24 h. Although , acute PCP did not change accuracy of responding or processing speed , repeated PCP revealed delayed deficits in cognitive processing speed which were partly ameliorated by clozapine . Extended inter-trial intervals increased premature responding , while SDA revealed that clozapine modified persistent PCP-induced deficits in lnBeta ( a composite measure of risk taking versus caution ) . Acute NMDA receptor antagonism impairs inhibitory control , whereas repeated treatment produces delayed deficits in cognitive processing speed . The ability of clozapine partially to restore persistent PCP-induced deficits in processing speed and in lnBeta is consistent with clinical findings . This suggests that the enduring effects of repeated PCP treatment , combined with SDA , offers a useful , translational , approach to evaluate novel cognitive enhancers in the 5-CSRTT . "
doc = nlp(text)
[(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]

In [ ]:
text1="The Italian Diabetes and Exercise Study 2 ( IDES-2 ) : a long-term behavioral intervention for adoption and maintenance of a physically active lifestyle . Physical activity ( PA ) /exercise have become an integral part of the management of type 2 diabetes mellitus ( T2DM ) . However , current guidelines are difficult to put into action in this population due to a number of barriers , especially the lack of acceptable , feasible , and validated behavioral intervention strategies . The present manuscript reports the rationale , study design and methods , and design considerations of the Italian Diabetes and Exercise Study ( IDES ) -2 , a randomized controlled trial testing the efficacy of a behavior change strategy in increasing total daily PA and reducing sedentary time ( SED-time ) in patients with T2DM . Starting 7 January 2014 , the IDES_2 began enrolling 300 patients with known T2DM of at least 1-year duration in three tertiary referral outpatient Diabetes Clinics in Rome . Additional requirements are age 40 to 80 years , body mass index 27 to 40 kg/m ( 2 ) , sedentary lifestyle , and physically inactive for at least 6 months , ability to walk 1.6 km without assistance , and eligibility after cardiovascular evaluation . Patients are randomized by center and within each center , by age and type of diabetes treatment to either the intervention or the control group . Patients in the intervention ( INT ) group ( n = 150 ) receive theoretical and practical exercise counseling consisting of aggregated behavior change techniques ( one individual theoretical counseling session plus eight twice-a-week individual theoretical and practical exercise counseling sessions ) once a year for 3 years . Patients in the control ( CON ) group ( n = 150 ) , receive standard care , including general physician recommendations for daily PA . The primary outcomes are total daily PA and SED-time , as measured objectively by the use of an accelerometer . Secondary outcomes include physical fitness , modifiable cardiovascular risk factors , musculoskeletal disturbances , well-being/depression , and health-related quality of life . The behavioral intervention strategy tested in the IDES_2 is based on solid theoretical grounds and uses several behavioral change techniques , two factors which were found to improve effectiveness of behavioral intervention . In addition , physicians and exercise specialists have been specifically trained for counselling/prescribing and supervising PA/exercise , respectively , in subjects suffering from metabolic disorders . Finally , the large sample size , the long study duration , and the objective measurement of PA allow statistically significant and scientifically robust conclusions to be drawn on the feasibility and efficacy of this intervention in T2DM patients . ClinicalTrials.gov ; NCT01600937 ; 10 October 2012 . "
text="Dissociation of acute and chronic intermittent phencyclidine-induced performance deficits in the 5-choice serial reaction time task : influence of clozapine . Cognitive deficits are a core feature of schizophrenia that respond minimally to existing drugs . PCP is commonly used to model schizophrenia-like deficits preclinically although different dosing protocols may affect different domains . Here we characterise the acute , and chronic intermittent effects of PCP in the 5-choice serial reaction time task ( 5-CSRTT ) in rats , and assess the effects of clozapine . In a novel approach , we also assess the effects of increased inhibitory load and conduct clinically relevant signal detection analysis ( SDA ) . The effects of acute and repeated PCP ( 2.58 mg/kg ) treatment on attentional processes and inhibitory control were assessed during and following the chronic treatment regime in the presence or absence of chronic clozapine ( 20 mg/kg/day ) . Thirty minutes post-PCP injection , there was an increase in anticipatory responding which disappeared after 24 h. Although , acute PCP did not change accuracy of responding or processing speed , repeated PCP revealed delayed deficits in cognitive processing speed which were partly ameliorated by clozapine . Extended inter-trial intervals increased premature responding , while SDA revealed that clozapine modified persistent PCP-induced deficits in lnBeta ( a composite measure of risk taking versus caution ) . Acute NMDA receptor antagonism impairs inhibitory control , whereas repeated treatment produces delayed deficits in cognitive processing speed . The ability of clozapine partially to restore persistent PCP-induced deficits in processing speed and in lnBeta is consistent with clinical findings . This suggests that the enduring effects of repeated PCP treatment , combined with SDA , offers a useful , translational , approach to evaluate novel cognitive enhancers in the 5-CSRTT . "
doc = nlp1(text1)
[(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]

[('type 2 diabetes mellitus', 240, 264, 'DISEASE'),
 ('T2DM', 267, 271, 'DISEASE'),
 ('strategies', 472, 482, 'DISEASE'),
 ('T2DM', 810, 814, 'DISEASE'),
 ('T2DM', 894, 898, 'DISEASE'),
 ('metabolic disorders', 2551, 2570, 'DISEASE'),
 ('T2DM', 2807, 2811, 'DISEASE')]